## Метрики в задаче регрессии 📏
Как гласит одна замечательная фраза: "Если ты не знаешь как что-то измерить, то ты не можешь это улучшить". Действительно, сложно спорить. Дело в том, что цель любой ML-модели - осуществлять верные предсказания, чтобы хорошо решать определенную бизнес задачу. Я рекомендую избегать результаты экспериментов, где просто говорят, что модель "хорошо" или "адекватно" работает. Что значит "хорошо" или "плохо"? Это сколько? 

Как видишь, такие термины лишены деталей, которые усложняют процесс интерпретации и улучшения модели. Вместо этого было бы идеально описывать результаты своих экспериментов фразами:
- "В среднем модель ошибается на 10.000 руб. при прогнозировании стоимости автомобиля"
- "В среднем модель ошибается на 15% от истинной цены услуги" 
- ...

Именно для этого и используются метрики.

- Метрика — это некоторая математическая формула, позволяющая оценить насколько хорошо модель выполняет свою задачу. Например, точность предсказаний или качество классификации.

Метрик в ML очень много, но в рамках этого ноутбука мы сфокусируемся на основных метркиах для решения задачи регрессии. Многие метрики уже имплементированы в sklearn в разделе [metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html)

In [1]:
import numpy as np
import pandas as pd
import warnings

SEED = 23
warnings.filterwarnings('ignore')

## Данные 📊
Я предлагаю использовать несложный датасет по оценке стоимостей автомобилей. Это неплохой вариант для нашей темы, так как целевая переменная будет содержать информацию о дешевых и дорогих автомобмилях, что отлично подходит для сравнения таких метрик как $MAE$ и $RMSE$. Датасет содержит информацию о ценах на автомобили, произведенные в США. Более подробно с датасетом можно ознакомиться [здесь](https://www.kaggle.com/datasets/hellbuoy/car-price-prediction?select=CarPrice_Assignment.csv).


In [2]:
data = pd.read_csv('../data/regression/usa_car_prices.csv')
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


- В датасете 26 фичей. Логично предположить, что не все из них важны и для построения лучше модели необходимо сделать отбор признаков - _feature selection_. В рамках нашей темы это будет избыточно, поэтому я просто сфокусируюсь на нескольких числовых признаках которые отберу по-своему предпочтению:
    - _Кол-во лошадиных сил | horsepower_
    - _Длина авто | carlength_
    - _Объем двигателя | enginesize_
    - _Вес авто_ | curbweight_
    - _Цена автомобиля | price_

In [3]:
# отбираем признаки
features = [
    'horsepower',
    'carlength',
    'enginesize',
    'curbweight',
    'price'
]

df = data[features]
df.head()

,horsepower,carlength,enginesize,curbweight,price
0,111,168.8,130,2548,13495.0
1,111,168.8,130,2548,16500.0
2,154,171.2,152,2823,16500.0
3,102,176.6,109,2337,13950.0
4,115,176.6,136,2824,17450.0


Давай обучим простую модель регресии и сравним прогнозы модели с известными значениями

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# как всегда, вначале разобьем данные на train и test
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, shuffle=True)

# осуществляем масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# обучим модель
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# спрогнозируем на test
y_pred = model.predict(X_test_scaled)

# cформируем таблицу с истинными и предсказанными значениями
metrics_results = pd.DataFrame({'true': y_test, 'pred': y_pred.flatten()})
metrics_results['error'] = metrics_results['true'] - metrics_results['pred']
metrics_results.head()

,true,pred,error
74,45400.0,33459.814874,11940.185126
51,6095.0,5441.606425,653.393575
46,11048.0,12783.570040,-1735.570040
14,24565.0,18848.847159,5716.152841
18,5151.0,205.239611,4945.760389


Отлично, теперь у нас есть ошибка на каждом объекте выборки. Однако, мы ведь не можем судить о качестве модели, используя лишь 1 объект? Необходимо учитывать все объекты, какие есть варианты?
- _"Суммирование отклонения_": не подходит, так как произойдет компенсация ошибки. Например, если отклонения -2 и 2, то сумма равна 0. Получается что ошибки нет, однако это не так
- _"Квадрат отклонения_": подходит, компенсации ошибки не не произойдет
- _"Абсолютное отклонение_": подходит, компенсации ошибки ткакже не происходит

In [5]:
# давай посмотрим какие ошибки мы получим, используя методы выше
sum_error = metrics_results['error'].sum().item()
sum_squared_error= (metrics_results['error'] ** 2).sum().item()
sum_abs_error = metrics_results['error'].abs().sum().item()

print(f"Сумма отклонений: {sum_error:.2f}")
print(f"Сумма квадратов отклонений: {sum_squared_error:.2f}")
print(f"Сумма абсолютных отклонений: {sum_abs_error:.2f}")

Сумма отклонений: 67038.65
Сумма квадратов отклонений: 1078337927.81
Сумма абсолютных отклонений: 178092.24


Полученные ошибки - накопительные, а нам бы желательно понимать как, например, модель ошибается в среднем, поэтому давай просто усредним полученные значения. 

In [6]:
n = metrics_results.shape[0]
mean_sum_error = sum_error / n
mean_squared_error = sum_squared_error / n
mean_abs_error = sum_abs_error / n

print(f"Средняя сумма отклонений: {mean_sum_error:.2f}")
print(f"Средняя сумма квадратов отклонений: {mean_squared_error:.2f}")
print(f"Средняя сумма абсолютных отклонений: {mean_abs_error:.2f}")

Средняя сумма отклонений: 1081.27
Средняя сумма квадратов отклонений: 17392547.22
Средняя сумма абсолютных отклонений: 2872.46


Теперь полученные метрики отражают <u>усредненную</u> ошибку модели для всех объектов выборки, то что нам нужно! Как мы обсудили выше - некорректо использовать "среднюю сумму отклонения" из-за компенсации. Оставшивеся 2 метрики действительно используются в машинном обучении и называются соответственно _среднеквадратичная_ и _среднеабсолютная_ ошибки о которых мы поговорим далее.

## Средняя квадратическая ошибка | Mean Squared Error | MSE
$$ MSE = \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

$MSE$ — одна из самых популярных метрик для регрессионных моделей. Она измеряет средний квадрат разницы между реальными значениями $y_i$ и предсказаниями $\hat{y}_i$.

Интуитивно, $MSE$ показывает, как сильно в среднем ошибается модель. Так как отклонения возводятся в квадрат, то штраф за ошибку квадратичный - 1:2. Давай взглянем на вид функциональной зависимсоти ошибки. Ну так как это квадратичная ошибка, то график должен быть похож на параболу

In [7]:
import plotly.express as px

# ошибки
metrics_results['error_squared'] = metrics_results['error'] ** 2

# сортировка чтобы красиво отрисовалось
metrics_results = metrics_results.sort_values("error")

fig = px.line(
    metrics_results,
    x="error",
    y="error_squared",
    title="Mean Squared Error",
    labels={"error": "Отклонение", "error_squared": "Ошибка"}
)

fig.add_scatter(
    x=metrics_results["error"], y=metrics_results["error_squared"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=800,
    title_text="Квадратичная ошибка | Squared Error",
    showlegend=True
)

Хмм, ну и где парабола? Да, график похож на нее, но почему правая чсть длинее левой? В чем может быть причина?

<!-- Ответ
Дело в том, что распределение цены авто не является нормальным/симметричным - есть небольшое кол-во наблюдений с большой ценой. Следовательно, это приводит к большим положительным отклонениям которые делают распределение ошибки несимметричным. Скорее всего распределение таргета имеет длинный правый хвост, на котором модель занижает прогноз. Если бы в выборке было достаточно дорогих авто, модель научилась бы лучше предсказывать их стоимость, и хвост распределения ошибок был бы более симметричным.
-->

In [8]:
# твои мысли 🧠

Чтобы убедиться, что ошибка действительно симметричная давай зафиксируем диапазон отклонения, например будем рассматривать только от $[-5.5k, 5.5k]$

In [9]:
error_threshold = 5500
squared_error_threshold = 30*10**6

metrics_results_cut = metrics_results[metrics_results['error'] <= error_threshold]
metrics_results_cut = metrics_results_cut[metrics_results_cut['error_squared'] <= squared_error_threshold]

fig = px.line(
    metrics_results_cut,
    x="error",
    y="error_squared",
    title="Mean Squared Error",
    labels={"error": "Отклонение", "error_squared": "Ошибка"}
)

fig.add_scatter(
    x=metrics_results_cut["error"], y=metrics_results_cut["error_squared"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=800,
    title_text="Квадратичная ошибка | Squared Error",
    showlegend=True
)

Вот теперь хорошо видно, что ошибка симметричаня. Заметь, что величина штрафа - нелинейная (квадртатичная). Выходит, чем сильнее ошибается модель, тем сильнее штраф. Отсюда мы встречаемся с первым минусом у $MSE$:

- $MSE$ очень чувствительна к выбросам!

Могу доказать на простом примере, смотри ...

In [10]:
# давай добавим новый столбец - абсолютное отклонение и квадаратичное
metrics_results['error_abs'] = metrics_results['error'].abs()

# найдем наблюдения с минимальным и максимальным абсолютными отклонениями
min_error_abs = metrics_results['error_abs'].min()
max_error_abs = metrics_results['error_abs'].max()

min_max_error_df = metrics_results[
    metrics_results['error_abs'].isin([min_error_abs, max_error_abs])
]

print(f"Минимальное абсолютное отклонение: {min_error_abs:.2f}")
print(f"Максимальное абсолютное отклонение: {max_error_abs:.2f}")

Минимальное абсолютное отклонение: 19.75
Максимальное абсолютное отклонение: 14495.02


Как видишь, в наших данных есть наблюдения где модель практически не ошибается - очень маленькое отклонение, а есть объекты на которых ошибка просто коллосальная. Если сравнить 2 ошики выше между собой, то получится, что одна ошибка в 763 раза больше другой. Это очень много! Но почему так происходит? Дело в том, что цены на авто распределены ненормально - большинство наблюдений (авто) имеют небольшую стоимость, но есть небольшое количество наблюдений с большой стоимостью - дорогие авто. Так как дорогих авто в обучающей выборке значительно меньше бюджетных, то у модели появляется некоторый _bias_ - она начинает занижать цену автомобилей.

In [11]:
# взглянем на распределение цен авто на test
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

# Distribution plot
# считаем оценку плотности для фактических и предсказанных значений
kde_test = gaussian_kde(y_test)

# определим диапазон значений по которым бдуем строить плотности
min_value = min(y_test.min(), y_test.min())
max_value = max(y_test.max(), y_test.max())

# также задаим сколько точек будем строить на графике
n_points = 200

# итоговый диапазон значений
x_vals = np.linspace(min_value, max_value, n_points)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_vals, y=kde_test(x_vals),
    fill='tozeroy', name='Известные', mode='lines',
    line=dict(color='blue'), opacity=0.6
))

fig.update_layout(
    title='Распределение цен на авто | Test Data',
    xaxis_title='Sales', yaxis_title='Плотность',
    width=1000, height=600
)

fig.show()

Можно рассматривать дорогие автомобили как выбросы, которые непропорционально влияют на критерий качества и обучение модели. В таких условиях стандартная линейная регрессия часто ведёт себя плохо на хвостах распределения. К счастью, эту проблему можно решить при помощи специальных методов (логарифмирование, использование устойчивых функций потерь, ...)

В рамках нашего урока, наблюдения выше идеально подходят для демонстрации чувствительности $MSE$ к выбросам. Так как модель очень сильно ошибается на втором объекте, то его можно смело посчитать за выброс. Наша исходная модель совершает вот таки квадартичные ошибки на данных объектах:

In [12]:
errors_old = min_max_error_df['error_squared'].values.tolist()
print('Ошибки:', errors_old)

Ошибки: [389.94176637864393, 210105511.76706648]


Тепреь предположим, что мы улучшили нашу модель - добавили больше признаков и качество симметрично улучшилось - стало точнее на 10$ (взял цифру с потолка <смеющийся смайлик>)

In [13]:
# красиво оформим DataFrame - как буто результат эксперимента модели A
columns = [
    'true',
    'pred',
    'error_squared',
    'error_abs'
]

model_a_results = min_max_error_df[columns].copy()
model_a_results.insert(0, 'model_version', 'A') # добавляем версию модели
model_a_results

,model_version,true,pred,error_squared,error_abs
157,A,7198.0,7217.746943,3.899418e+02,19.746943
128,A,37028.0,22532.983209,2.101055e+08,14495.016791


In [14]:
# результат эксперимента модели B
model_b_results = pd.DataFrame({
    'model_version': ['B', 'B'],
    'true': [7198.0, 37028.0],
    'pred': [7207.7, 22542.9], # предсказание теперь точнее на 10$ для обоих объектов
})

# считаем квадратичную и абсолютные ошибки
model_b_results['error_squared'] = (model_b_results['true'] - model_b_results['pred']) ** 2
model_b_results['error_abs'] = (model_b_results['true'] - model_b_results['pred']).abs()
model_b_results

,model_version,true,pred,error_squared,error_abs
0,B,7198.0,7207.7,9.409000e+01,9.7
1,B,37028.0,22542.9,2.098181e+08,14485.1


In [15]:
# теперь посчитаем на сколько уменьшилась квадратичная и абсолютная ошибки
error_reduction_squared = model_a_results['error_squared'].values - model_b_results['error_squared'].values
error_reduction_abs = model_a_results['error_abs'].values - model_b_results['error_abs'].values
print('Изменение квадратичной ошибки: ', error_reduction_squared.tolist())
print('Изменение абсолютной ошибки: ', error_reduction_abs.tolist())

Изменение квадратичной ошибки:  [295.8517663786475, 287389.75706651807]
Изменение абсолютной ошибки:  [10.04694321606894, 9.91679085148644]


Видно, что сильнее всего (почти на 300к) изменилась ошибка на выбросе (второе наблюдение), а не на "нормальном" объекте. Следовательно, модели будет всегда выгоднее точнее предсказывать выброс, ценой большей ошибки на "нормальных" данных. Таким образом, модель будет всегда подстраиваться под выбросы, чтобы достичь наименьшего $MSE$. Как думаешь хорошо это или плохо?

<!-- Ответ
Это зависи от бизнес задачи. Необходимо задать вопрос бизнесу - "Важно ли для нас хорошо предсказывать цену на выбросах или важна стабильная работа модели на основном сегменте цен?". Поэтому далее возможны 2 сценария
- Вариант А: Настраиваться на выбросы
    - Лучше будет предсказывать редкие наблюдения, однако хуже будет прогноз на основных наблюдениях - скорее всего будет завышать прогноз
- Вариант B: Игнорировать выбросы (их удаления или использование устойчивых функций потерь)
    - Модель будет ошибаться на выбросах - не сможет хорошо их предсказать - скорее всего будет занижать прогноз, однако на основных наблюдениях модель будет точна
-->

In [16]:
# твои мысли 🧠

А что с абсолютной ошибкой? Как видишь, для этой метрики модели не важно на каком объекте улучшать прогноз, так как итоговые ошибки равны. Поэтому $MAE$ устойчив к выбросам! Давай перейдем к следующим 2-м минусам метрики $MSE$, связанные с еденицами измерения и интерпретируемостью.

#### _Проблема квадратных значений_

Так как значение ошибки возводится в квадрат, то при интерпретации ошибки мы будем оперировать квадратными еденицами - квадратные доллары/рубли/штуки, ... Представь, ты обучил модель, а затем говоришь заказчику, что твоя модель в среднем ошибается на 5000 квадратных долларов. Не совсем понятно как это понимать, не так ли? 

Избавить от квадрата очень просто - извелкаем корень из итоговой ошибки (спасибо курсу школьной математики <смайл смех>) и все, теперь квадратные доллары  это снова привычные и понятные нам доллары

#### _Проблема: ошибка в контексте - большая/маленькая?_

Также при расчете $MSE$ и последующего извлечения корня мы получаем ошибку значение которой не дает информации о критичности - большая или маленькая? Например, получил ты $RMSE = 5000$. Это много или мало? Один из вариантов это сбегать к заказчику и спросить. А если он в отпуске или не отвечает? Как тогда поступить? К счастью, можно учесть масштаб целевой переменной и получить новую метрику - "коэффициент детерминации R2" который легко интерпретируется.

## Корень из средней квадратичной ошибки | Root Mean Squared Error | RMSE
Мы уже затронули данную метрику выше - это просто квадратный корень из среднеквадратичной ошибки _MSE_

$$ RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

В отличие от $MSE$, величина $RMSE$ выражается в тех же единицах, что и целевая переменная. Это делает её более интерпретируемой: например, если мы предсказываем стоимость автомобилей в долларах, то $RMSE$ показывает среднюю ошибку прогноза в  настоящих долларах, вместо квадратных. Однако важно помнить, что само значение $RMSE$ не всегда говорит, много это или мало. Чтобы правильно оценить качество модели, необходимо сопоставлять $RMSE$ с разбросом целевой переменной - использовать метрику $R2$

$RMSE$ отличается от $MSE$ только тем, что в него добавлен квадратный корень. Метрика унаследовала все недостатки $MSE$, но при этом имеет одно важное преимущество — величина ошибки выражается в тех же единицах, что и сама целевая переменная $y$

##  Логарифм средней квадратичной ошибки | Mean Squared Logarithmic Error | MSLE
Является вариантом $MSE$, которая применяется к логарифмам предсказанных и фактических значений. Основная цель MSLE — оценка моделей, где важнее относительная ошибка, а не абсолютная. Она особенно полезна, когда значения таргета могут сильно различаться по масштабу, и ошибки на больших значениях не должны доминировать над ошибками на малых.

$$\text{MSLE} = \frac{1}{n} \sum_{i=1}^{n} \Big(\ln(1 + \hat{y}_i) - \ln(1 + y_i)\Big)^2$$

Данная метрика идеально подходит, когда необходимо предсказывать не точное значение таргета, а его порядок. Натуральный логарифм в формуле как раз этим и занимается - переводит значение в другую шкалу - шкалу порядков на основании которой и происходит вычисление ошибки. Также хочу отметить, что $MSLE$ слабее штрафует в сравнении с $MSE$ за счет использования натурального логарифма.

In [ ]:
# закодим метрику
def mean_squared_log_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2)

## Коэффициент Детерменации $R2$
Это одна из наиболее популярных метрик качества регрессионных моделей. Как мы уже выясниили ранее, _RMSE_ не дает оценку критичности ошибки - большая или маленькая, поэтому если нормировать _"MSE"_ на дисперсию целевой перменной $y$, то получим абсолютно новую мерику - коэффициент детерменации, глядя на который можно легко понять на сколько хороша наша модель, получается своего рода нормализованный $RMSE$

R2 показывает, как хорошо модель объясняет дисперсию целевой переменной $y$ 

$$ R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$

- Числитель: используется не _MSE_, а _SSE - Sum Squared Error_, но интуация +/- как с _RMSE_
- Знаменатель: используется дисперсия таргета $y$, еще называют _SST - Sum of Squeres Total_ - общая сумма квадратов

Выходит, что в идеале модель должна предсказывать либо идеально значения таргета, либо чтобы _SSE_ равнялся дисперсии $y$. Значение $R^2$ всегда лежит в интервале от $(-\infty, 1]$, так как это нормированная метрика. А вот так можно интерпретировать полученные результаты:
- $R^2 = 1$ - модель идеально предсказывает данные, без ошибок (редко бывает на практике)
- $R^2 = 0$ - модель ничем не лучше прогноза "средним" - "глупая модель" (бывает на практике, необходимо избегать)
- $R^2 < 0$ - модель хуже прогноза "средним" (бывает на практике, необходимо избегать)

На практике $R2$ для "разумной" модели обычно лежит на отрезке от $[0, 1]$. Чем ближе к 1 тем лучше! Если $R^2 < 0$, то с моделью 100% что то не так - недообучение/переобучение, выбросы, возможно _train_ и _test_ имеют разные распределения, ... В таких случаях необходим детальный анализ данных и ошибок модели. Хочу отметить, что формулу выше можно переписать в более интерсном виде:


$$ R^2 = 1 - \frac{SSE}{SST} = 1 - \frac{n\cdot RMSE^2}{SST}, RMSE = \sqrt{\frac{SSE}{n}}$$

$$R^2 = 1 - \left( \frac{RMSE_\text{model}}{RMSE_{\text{baseline}}} \right)^2, RMSE_{\text{baseline}} = \sqrt{\frac{\mathrm{SST}}{n}}$$

Фомулы выше доказывают, что $R2$ можно интерпретировать как показатель того, насколько модель предсказывает лучше, чем простая стратегия прогнозирования средним - часто используемая в качестве "baseline" решения.

In [17]:
# закодим метрику R2 и оценим качество нашей модели
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot) if ss_tot != 0 else 0

r2_score_test = r2_score(y_true=metrics_results['true'], y_pred=metrics_results['pred'])
print('Коэффициент детерминации R2 | Test:', r2_score_test)

Коэффициент детерминации R2 | Test: 0.8101000960701704


Получили довольно хороший результат, наша модель на 80% объясняет дисперсию целевой переменной. Лишь 20% отсутствует - можно подумать как усложнить модель. 

## Минусы
Идеальных метрик не бывает, у каждой найдутся минусы. Вот сейчас об этом и поговорим:
- *Не отражает величину ошибки*
    - Показывает долю дисперсии целевой переменной которая объясняет модель. Однако, большой $R2$ еще не гарантирует маленькую $RMSE$. Необходимо дополнительно валидировать $RMSE$

- *Чувствителен к выбрасам*
    - Так как метрика условно основана на $RMSE$, то наследует проблему с выбросами. Даже наличие нескольких выбросов может значительно "уронить" метрику

- *Зависит от числа признаков*
    - Никогда не уменьшается при добавлении новых признаков, даже если они бесполезны. В этом случае необходимо использовать скорректированный $R2$

- _Зависит от дисперсии целевой переменной_
    - Если дисперсия таргета $y$ маленькая, то даже модель с хорошим $RMSE$ даст низкий $R2$, и наоборот, при большой дисперсии таргета даже не очень точная модель может показать высокий $R2$

- _Зависит от налиичия intercept в модели (линейные модели)_ 
    - Если модель не включает _intercept_ (свободный член), то интерпретация $R2$ рушится и вводит в заблуждение, так как нарушается предположение $\sum_{i=1}^{n} \left(y_i - \hat{y}_i\right) = 0$

## Почему R2 зависит от количества фичей?
Хочу обратить твое внимание на смаую значимую проблему метрики $R2$ - зависимость от количества признаков. Если ты получил большое значение $R2$ и использовал много признаков, то не факт, что твоя модель действительно "выучила" что-то полезное. Добавление новых признаков никогда не уменьшает метрику, а даже увеличивает! Рузумеется, все эти утверждения относятся только к качеству на _train_ или _validation_, на _test_ мы будем видеть ухудшение метрики? Но ориентироваться на _test_ - нельзя, так как в этом случае ты будешь переобучаться на него. Если ты мне не веришь, то я могу доказать, смотри <эмодзи глаза>


Напомню, что изначально мы обучили модель, используя 4 признака. Модель получила довольно неплохой результат на _test_: $R2 = 0.81$. Фичи скорее всего информативные и имеют значимость при прогнозировании. Давай убедимся в этом. Есть идеи как это можно сделать?

<!-- Ответ
Есть несколько способов:
1. Проверить корреляцию с таргетом
2. Посмотреть на их важность в итоговой обученной модели
-->

In [18]:
# твои мысли 🧠

In [19]:
# проверим корреляцию с таргетом - "price"
features_corr = data[features].corr()['price'].sort_values(ascending=False)[1:]

# важность в модели | "Feature Importance"
features_imp  = pd.DataFrame({
    'feature': features[:-1],
    'importance': model.coef_
})

features_summary_df = features_imp.merge(features_corr, left_on='feature', right_index=True)
features_summary_df

,feature,importance,price
0,horsepower,1755.758012,0.808139
1,carlength,281.319695,0.682920
2,enginesize,2178.328811,0.874145
3,curbweight,2632.272265,0.835305


Отлично, признаки значимы и имеют хорошую линейную зависимость с таргетом, а значит действительно важны. Теперь давай сгенерируем N-бесполезных признаков (шум) и посмотрим как будет изменяться $R2$ от их количества на обучающей выборке - _train_. Напомню, что ранее мы уже разбили данные на _train_ и _test_

In [20]:
NOISY_FEATURES_COUNT = 30
np.random.seed(SEED)

X_train_aug = X_train_scaled.copy()
X_test_aug = X_test_scaled.copy()

scores_train = []
scores_test = []

for i in range(NOISY_FEATURES_COUNT):
    noise_train = np.random.randn(X_train.shape[0], 1)
    noise_test = np.random.randn(X_test.shape[0], 1)
    
    X_train_aug = np.hstack([X_train_aug, noise_train])
    X_test_aug = np.hstack([X_test_aug, noise_test])
    
    _model = LinearRegression()
    _model.fit(X_train_aug, y_train)

    y_pred_train = _model.predict(X_train_aug)
    y_pred_test = _model.predict(X_test_aug)

    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    
    scores_train.append(r2_train)
    scores_test.append(r2_test)

In [21]:
import plotly.graph_objects as go

features_count = list(range(1, NOISY_FEATURES_COUNT + 1))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_train, 
    mode='lines+markers', 
    name='R² | Train',
    line=dict(color='blue'),
    marker=dict(size=6)
))


fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_test, 
    mode='lines+markers', 
    name='R² | Test',
    line=dict(color='red'),
    marker=dict(size=6)
))

fig.update_layout(
    height=700, width=1500,
    title="Влияние шумовых фичей на R²",
    xaxis_title="Количество шумовых фичей",
    yaxis_title="R²",
)

fig.show()

Что и требовалоьс доказать <смеющийся смайл>. Давай разберемся почему так происходит! Для начала необходимо взглянуть на формулу $R2$

$$
R^2 = 1 - \frac{SSE}{SST} = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$

На формуле хорошо заметно, что $SST$ зависит только от данных, а значит будет фиксирован. Выходит, что $R2$ будет только зависеть от $SSE$ - чем меньше, тем лучше! При добавлении новых фичей модель становится более гибкой - пространство допустимых решений (гиперплоскостей регрессии) расширяется - значит новая модель может минимум не хуже, чем старая, аппроксимировать данные. Следовательно, ошибка на обучающей выборке либо уменьшится, либо останется прежней:

$$
SSE_{\text{new}} \leq SSE_{\text{old}}
$$

Вот именно поэтому добавление новых признаков, даже неинформативынх, приводит к росту $R2$, так как модель начинает учить шум и переобучаться на обучающую выборку. Привер выше наглядо демонстрирует данную проблему - я добавил 30 шумовых признаков, которые +/- симметрично увеличили качество на _train_ и снизили на _test_ на 5%. К счастью, данная проблема решается за счет использования скорректированного $R2$

## Почему R2 так называется?
Не знаю как ты, а вот я при первой встрече с этой мерикой задумался почему она называется "р-квадрат"? Что такое "Р" и почему в квадрате? Оказывается, что это идет из статистики и связано с корреляцией Пирсона. Смотри, корреляция обычно высчитывается вот по такой формуле:

$$
r = \frac{\text{Cov}(X, Y)}{\sigma_X \, \sigma_Y}
$$

При построении модели линейной регресии возникает вопрос: "Как хорошо предсказанные значения $\hat{y}$ коррелируют с фактическими $y$?". Оказывается, что если возвести корреляцию в квадрат, то как раз получим метрику, которая позволяет объяснить дисперсию $y$ через $\hat{y}$

$$
R^2 = r^2(y, \hat{y})
$$ 

Поэтому метрика получила название «$R^2$» — квадрат коэффициента множественной корреляции. 

## Скорректированный коэффициент детерменации $R2$
Аналогичен $R2$, за тем исключением, что метрика учитывает колечество используемых признаков в модели:

$$
R_{\text{adj}}^2 = 1 - \frac{n-1}{n-p-1} \, (1 - R^2)
$$

- $R^2$ — обычный коэффициент детерминации
- $n$ — количество наблюдений
- $p$ — число признаков/фич в модели

Идея проста -  метрика «штрафует» модель за каждый используемый признак, поэтому если признак действительно полезный, то его вклад будет больше чем штраф за его добавление. 

In [22]:
# метрику можно легко закодить
def r2_score_adj(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n - 1) / (n - p - 1) if n - p - 1 > 0 else 0

# теперь сравнив R2 и R2-скорректированный
r2_score_test = r2_score(
    y_true=metrics_results['true'],
    y_pred=metrics_results['pred']
)

r2_score_adj_test = r2_score_adj(
    y_true=metrics_results['true'],
    y_pred=metrics_results['pred'],
    n=len(metrics_results),
    p=model.coef_.shape[0]
)

print(f'Коэффициент детерминации R2 | Test: {r2_score_test:.4f}' )
print(f'Коэффициент детерминации R2-скорректированный | Test: {r2_score_adj_test:.4f}')

Коэффициент детерминации R2 | Test: 0.8101
Коэффициент детерминации R2-скорректированный | Test: 0.7968


$R^2_{adj}$ на _test_ теперь выглядит менее оптимистичнее, что мы и ожидали. Теперь давай проведем аналогичный эксперимент с шумовыми фичами и посмотрим какие результаты мы получим

In [23]:
NOISY_FEATURES_COUNT = 30
np.random.seed(SEED)

X_train_aug = X_train_scaled.copy()
X_test_aug = X_test_scaled.copy()

scores_train_adj = []
scores_test_adj = []

for i in range(NOISY_FEATURES_COUNT):
    noise_train = np.random.randn(X_train.shape[0], 1)
    noise_test = np.random.randn(X_test.shape[0], 1)
    
    X_train_aug = np.hstack([X_train_aug, noise_train])
    X_test_aug = np.hstack([X_test_aug, noise_test])
    
    _model = LinearRegression()
    _model.fit(X_train_aug, y_train)

    y_pred_train = _model.predict(X_train_aug)
    y_pred_test = _model.predict(X_test_aug)

    r2_adj_train = r2_score_adj(
        y_true=y_train, y_pred=y_pred_train, n=len(y_train), p=_model.coef_.shape[0]
    )

    r2_adj_test = r2_score_adj(
        y_true=y_test, y_pred=y_pred_test, n=len(y_test), p=_model.coef_.shape[0]
    )

    scores_train_adj.append(r2_adj_train)
    scores_test_adj.append(r2_adj_test)

In [24]:
features_count = list(range(1, NOISY_FEATURES_COUNT + 1))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_train_adj, 
    mode='lines+markers', 
    name='​R² adj | Train',
    line=dict(color='blue'),
    marker=dict(size=6)
))


fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_test_adj, 
    mode='lines+markers', 
    name='R² adj | Test',
    line=dict(color='red'),
    marker=dict(size=6)
))

fig.update_layout(
    height=700, width=1500,
    title="Влияние шумовых фичей на скорректированный R²",
    xaxis_title="Количество шумовых фичей",
    yaxis_title="R² скорректированный",
)

fig.show()

Использование скорректированного $R2$ сильнее отражает эффект переобучения, особенно на тестовой выборке! Теперь давай финально сравним ​$R^2$ vs ​$R^2_{adj}$ на _train_ и _test_ на 1 графике

In [25]:
features_count = list(range(1, NOISY_FEATURES_COUNT + 1))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_train, 
    mode='lines+markers', 
    name='​R² | Train',
    marker=dict(size=6)
))


fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_train_adj, 
    mode='lines+markers', 
    name='R² adj | Train',
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_test, 
    mode='lines+markers', 
    name='R² | Test',
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=features_count, 
    y=scores_test_adj, 
    mode='lines+markers', 
    name='R² adj | Test',
    marker=dict(size=6)
))

fig.update_layout(
    height=700, width=1500,
    title="Влияние шумовых фичей на скорректированный R²",
    xaxis_title="Количество шумовых фичей",
    yaxis_title="R² скорректированный",
)

fig.show()

- Данный график наглядно показывает разницу поведения метрик на обучающей и тестовой выборках. Особенно хорошо это заметно на train: если обычный $R^2$ продолжает расти при добавлении шумовых признаков, то скорректированный $R_{\text{adj}}^2$ в среднем снижается за счёт встроенного штрафа за избыточные признаки.

## Средняя абсолютная ошибка | Mean Absolute Error | MAE
На самом деле ммы уже встречались с данной метрикой в самом начале, когда высчитывали абсолютные отклнения. $MAE$ показывает, насколько в среднем предсказанные значения отклоняются от фактических по модулю.

$$
MAE = \frac{1}{n}\sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

Так как теперь отклонения не возводятся в квадрат, то штраф за ошибку линейный - 1:1. Давай взглянем на вид функциональной зависимсоти ошибки:

In [26]:
fig = px.line(
    metrics_results,
    x="error",
    y="error_abs",
    title="Mean Absolute Error",
    labels={"error": "Отклонение", "error_abs": "Абсолютная ошибка"}
)

fig.add_scatter(
    x=metrics_results["error"], y=metrics_results["error_abs"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=800,
    title_text="Абсолютная ошибка | Absolute Error",
    showlegend=True
)

Как и в случае $MSE$, правая часть больше левой. Мы уже выяснили, что это происходит из-за несимметричного таргета - есть небольшое количество дорогих авто на которых модель очень сильно ошибается. Для симметричности ошибки необходимо зафиксировать диапазон отклонений, например от $[-5.5k, 5.5k]$ и затем сравним $MAE$ с $MSE$ 

In [27]:
fig = go.Figure()

metrics_results_cut['error_abs'] = metrics_results['error'].abs()

# MAE
fig.add_trace(go.Scatter(
    x=metrics_results_cut["error"],
    y=metrics_results_cut["error_abs"],
    mode="lines",
    name="MAE"
))

fig.add_trace(go.Scatter(
    x=metrics_results_cut["error"],
    y=metrics_results_cut["error_abs"],
    mode="markers",
    name="MAE points"
))

# MSE
fig.add_trace(go.Scatter(
    x=metrics_results_cut["error"],
    y=metrics_results_cut["error_squared"],
    mode="lines",
    name="MSE"
))

fig.add_trace(go.Scatter(
    x=metrics_results_cut["error"],
    y=metrics_results_cut["error_squared"],
    mode="markers",
    name="MSE points"
))

fig.update_layout(
    height=800, width=1000,
    title_text="Ошибки моделей: MAE vs MSE",
    xaxis_title="Отклонение",
    yaxis_title="Величина ошибки",
    showlegend=True
)

fig.show()


Функция ошибки для MAE выглядит как симметричная галочка — прямые под углом 45° по обе стороны от нуля. На графике выше удобно сравнивать штрафы для $MSE$ и $MAE$:
- Xорошо видно, что $MSE$ сильнее наказывает за большие отклонения

Однако, тот факт, что $MAE$ устойчив к выбросам - не означает что теперь необходимо использовать только эту метрику. Крупные ошибки могут быть важны, поэтому в этом случае лучше всего сработает $MSE$. Также у данной метрики есть недостаток, связанный с производной - функция ошибки не дифференцируется в нуле, и градиентные методы перестают работать, нона практике используют "хитрости", решающие эту проблему:

- Вычисляют "субградиенты"
    - В нуле берут множество различных градиентов на отрезке $[-1, 1]$. Оптимизация работает, но медленнее и менее устойчива, чем для $MSE$

- Применение сглаживаний
    - Заменяют $MAE$ на различиные приближения - $Huber loss$, $Smooth L1 loss$ ...

- Не градиентный методы
    - Например, линейное программирование или методы оптимизации, специально работающие с кусочно-линейными функциями.

## Средняя абсолютная ошибка в процентах | Mean Absolute Percentage Error | MAPE
Зачастую на практике важнее не абсолютная величина ошибки, а её размер в процентах - то есть на сколько мы ошибаемся <u>относительно</u> самого фактического значения:

$$
MAPE = \frac{1}{n} \sum_{i=1}^n \frac{|y_i - \hat{y}_i|}{|y_i|}  100\%
$$

Для хорошего понимая предлагаю рассмотреть вот такой пример. Допустим, ты решил построить модель, предсказывающую спрос на букеты роз в цветочном магазине. Если твоя модель прогнозирует 1 букет, а на спрос был 10 букетов, то это не тоже самое, что предсказать 1000, а продать 1009. Если замерить абсолютную ошибку, то она будет мала и одинакова для 2-х случаев - всего 9 букетов. 

Однако, если эту величну взять относительно истинного значения, то ужас! В первом случае ты ошибся в 10 раз, а во втором совсем чуть-чуть. На практике, данну метрику переводят в проценты путем умножения $MAPE$ на 100

In [28]:
# закодим метрику
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return 100 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + 1e-8))

# рассчитаем MAPE для нашего игрушечного примера
true = np.array([10, 1009, 0])
pred = np.array([1, 1000, 9])

abs_error = np.abs(true - pred)
print('Абсолютная ошибка:', abs_error.tolist())

abs_pct_error = (abs_error / true) * 100
print('Относительная абсолютная ошибка: ', abs_pct_error.tolist())

mape = mean_absolute_percentage_error(true, pred)
print('Средняя относительная абсолютная ошибка в % (MAPE): ', mape)


Абсолютная ошибка: [9, 9, 9]
Относительная абсолютная ошибка:  [90.0, 0.8919722497522299, inf]
Средняя относительная абсолютная ошибка в % (MAPE):  30000000030.29732


- Выходит, что в первом случае модель ошибается на 90%, во втором на ~0.9%, а в 3-ем на бесконечность. Я специально добавил ноль, чтобы продемонстрировать неопределенность метрики.

Теперь давай взглянем на функциональную зависимость данной метрики. Для симметричности ошибки необходимо зафиксировать диапазон отклонений, например от $[-5.5k, 5.5k]$

In [29]:
# добавим относительную ошибку (percentage error)
metrics_results_cut['error_pct'] = (metrics_results_cut['error_abs'] / metrics_results_cut['true']) * 100

fig = px.line(
    metrics_results_cut,
    x="error",
    y="error_pct",
    title="Mean Absolute Percentage Error",
    labels={"error": "Отклонение", "error_pct": "Относительная ошибка, %"}
)

fig.add_scatter(
    x=metrics_results_cut["error"], y=metrics_results_cut["error_pct"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=1000,
    title_text="Mean Absolute Percentage Error",
    showlegend=True
)

- График хорошо демонстрирует несиммеричность ошибки. Есть отклонения, на которых относительная ошибка становится очень большой

## Минусы MAPE
Я бы не рекомендовал использовать $MAPE$ на начальных этапах построения модели, так как метрика очень чувствительна как к прогнозам, так и к известным значениям таргета. Очень часто проблема новичков - берут метрику и получают просто космические ошибки в процентах или отрицательные значения. Так не пойдет, необходимо разбираться ... 

- *Чувствительность к нулевым и малым значениям таргета*
    - Если есть хоть один ноль в таргете, то $MAPE$ автоматически становится неопределен. 
    - Малые значения таргета сильно "раздувают" ошибку, что создает впечатления "ужасной" модели. На обучении модель "понимает", что лучше недопредсказывать малые значения таргета, что ведет к bias.

- *Смещенность*
    - Метрика может быть смещена в прогнозы, которые чаще оказываются меньше реальных значений - систематически недопрогнозирует. Будет казаться, что модель работает лучше, чем она есть на самом деле.

- *Несимметричность*
    - Метрика сильнее штрафует за "перепрогноз" - "строже" оценивает недооценку, чем переоценку, что делает её нессиметричной относительно фактического значения:

        - Перепрогноз: $y = 1$, $\hat{y} = 2$, $error_{abs} = 1$, $MAPE = 100\%$

        - Недопрогноз: $y = 2$, $\hat{y} = 1$, $error_{abs} = 1$, $MAPE = 50\%$

Давай создадим график на котором можно будет легко оценивать влияние недопрогноза и перепрогноза для конкрентного значения целевой переменной  $y$

In [30]:
# фиксируем фактическое значение таргета
y_true = 50

# формируем предсказания
min_value, max_value = 0, y_true + y_true + 50
n_points = 31

y_pred = np.linspace(min_value, max_value, n_points)
error_pct = 100 * np.abs(y_pred - y_true) / np.abs(y_true)

# создаем DataFrame для отрисовки
error_df = pd.DataFrame({
    "pred": y_pred,
    "error_pct": error_pct
}).sort_values("pred")

# чтобы сохранить порядок категорий в Plotly, переведём pred в строку
error_df['pred_str'] = error_df['pred'].round(2).astype(str)

fig = px.bar(
    error_df,
    x="error_pct",
    y="pred_str",
    orientation='h',
    title=f"MAPE при фиксированном таргете | y = {y_true}",
    labels={"pred_str": "Прогноз (ŷ)", "error_pct": "Относительная ошибка, %"},
    text="error_pct"
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

fig.update_layout(
    height=800,
    width=1000,
    yaxis={"categoryorder": "array", "categoryarray": error_df['pred_str'].tolist()},
    margin=dict(l=140)
)

fig.show()


На графике выше отлично видно как перепрогноз влияет на значение $MAPE$, особенно заметна несимметричость при малых значениях таргета $y$. Также данную ошибку удобно рассматривать от в разрезе таргета, что понять накаких объектах модель ошибается сильнее всего.

In [31]:
# необходимо для Plotly
_metrics_results_cut = metrics_results_cut.copy()
_metrics_results_cut.sort_values("true", inplace=True)
_metrics_results_cut['true'] = metrics_results_cut['true'].astype(str)

# исключаем NaN если вдруг есть нули в таргете
_metrics_results_cut = _metrics_results_cut.replace([np.inf, -np.inf], np.nan).dropna(subset=['error_pct'])

# рисуем
fig = px.bar(
    _metrics_results_cut,
    x="error_pct",
    y="true",
    orientation='h',
    title="Mean Absolute Percentage Error",
    labels={"true": "Истинное значение", "error_pct": "Относительная ошибка, %"},
)

fig.update_layout(
    height=700, width=1000,
)

fig.show()

На графике видно, что есть объект на котором $MAPE$ достигает почти 100%. Подумай, почему это плохо и как можно решить эту проблему?

<!-- Ответ
Усреднение довольно сильно исказит MAPE из-за выброса в 100%. Поэтому можно просто рассмотреть медиану, которая как известно из статистики устойчива к выбросам!
-->

In [32]:
# твои мысли 🧠

In [33]:
# расчитаем относительную ошибку в %
metrics_results['error_pct'] = metrics_results['error_abs'] / metrics_results['true'].abs() * 100
mape = metrics_results['error_pct'].mean()
median_ape = metrics_results['error_pct'].median()

print(f'Mean APE -> {mape:.2f}%')
print(f'Median APE -> {median_ape:.2f}%')

Mean APE -> 19.04%
Median APE -> 15.95%


## Симметричная средняя абсолютная ошибка в процентах | Symmetric Mean Absolute Percentage Error | SMAPE
$SMAPE$ — это модификация метрики $MAPE$, которая устраняет её ключевой недостаток — несимметричность при недооценке и переоценке прогнозов. В отличие от $MAPE$, где ошибка нормируется только на фактическое значение, $SMAPE$ использует среднее арифметическое между фактом и прогнозом как базу для нормировки:

$$
SMAPE = \frac{100\%}{n} \sum_{i=1}^{n} 
\frac{|y_i - \hat{y}_i|}{\frac{|y_i| + |\hat{y}_i|}{2}}
$$

Это трюк позволяет метрике:
- Быть симметричной — одинаково оценивать как перепрогноз, так и недопрогноз
- Иметь фиксированный предел — от 0% до 200%, можно иметь нули в таргете!
- Сохранять интерпретацию для удобного сравнения качества моделей

In [34]:
# закодим метрику
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return 100 * np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred)) / 2 + 1e-8))

# рассчитаем MAPE для нашего игрушечного примера
true = np.array([10, 1009, 0])
pred = np.array([1, 1000, 9])

abs_error = np.abs(true - pred)
print('Абсолютная ошибка:', abs_error.tolist())

abs_pct_error = (abs_error / ((true + pred)/2) * 100)
print('Относительная абсолютная ошибка: ', abs_pct_error.tolist())

smape = symmetric_mean_absolute_percentage_error(true, pred)
print('Симметричная средняя относительная абсолютная ошибка в % (SMAPE): ', smape)

Абсолютная ошибка: [9, 9, 9]
Относительная абсолютная ошибка:  [163.63636363636365, 0.8959681433549029, 200.0]
Симметричная средняя относительная абсолютная ошибка в % (SMAPE):  121.5107770125815


Теперь видно, что бесконечности больше нет, и итоговая метрика стала ниже. Можно заметить, что в знаменателе всё ещё возможен ноль. Да, это так — но в таком случае ситуация трактуется как отсутствие ошибки, ведь прогноз будет совпадает с фактом. Также хочу отметить, что если таргет равен нулю, а прогноз нет, то итоговая ошибка будет равна 200%, вот доказательство:

$$
\text{item}_i = \frac{2 \cdot |0 - \hat{y}_i|}{|0| + |\hat{y}_i|} = \frac{2 \cdot |\hat{y}_i|}{|\hat{y}_i|} = 2 \cdot 100\% \Rightarrow 200\%
$$

Давай сравним как ведут себя $MAPE$ и $SMAPE$ на фиксированном значении таргета

In [39]:
from plotly.subplots import make_subplots

# фиксируем фактическое значение таргета
y_true = 50

# формируем предсказания
min_value, max_value = 0, y_true + y_true + 50
n_points = 31

y_pred = np.linspace(min_value, max_value, n_points)

# высчитываем ошибки 
error_pct = 100 * np.abs(y_pred - y_true) / np.abs(y_true)
error_pct_sym = 100 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) / 2)

# необходимо для визуализации
error_df = pd.DataFrame({
    "pred": y_pred,
    "error_pct": error_pct,
    "error_pct_sym": error_pct_sym
}).sort_values("pred")

error_df['pred_str'] = error_df['pred'].round(2).astype(str)

# визуализируем
fig = go.Figure()

fig.add_trace(go.Bar(
    x=error_df["error_pct_sym"],
    y=error_df["pred_str"],
    orientation='h',
    name='SMAPE',
    text=error_df["error_pct_sym"].round(2),
    textposition='outside'
))

fig.add_trace(go.Bar(
    x=error_df["error_pct"],
    y=error_df["pred_str"],
    orientation='h',
    name='MAPE',
    text=error_df["error_pct"].round(2),
    textposition='outside'
))

fig.update_layout(
    barmode='group',
    title=f"SMAPE vs MAPE при фиксированном таргете | y = {y_true}",
    xaxis_title="Относительная ошибка, %",
    yaxis_title="Прогноз (ŷ)",
    yaxis={"categoryorder": "array", "categoryarray": error_df['pred_str'].tolist()},
    height=800,
    width=1000,
    margin=dict(l=140)
)

fig.show()

График идеально демонстрирует различную величину штрафа в метриках. 

## Взвешенная средняя абсолютная ошибка в процентах | Weighted Mean Absolute Percentage Error | WMAPE
Оказывается, что у метрики $MAPE$ есть модификация в виде взвешивания, которая позволяет не искажать метрику на малых значениях таргета:

$$
WMAPE = \frac{\sum_{i=1}^{n} |y_i - \hat{y}_i|}{\sum_{i=1}^{n} |y_i|} 
$$

То есть мы считаем отношение общей суммы ошибок к общей сумме фактических значений - это позволяет взвешивать ошибки пропорционально величине фактического значения из-за чего метрика не искажается:

In [36]:
# закодим метрику
def weighted_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return 100 * np.sum(np.abs(y_pred - y_true)) / (np.sum(np.abs(y_true)) + 1e-8)

# рассчитаем WMAPE и MAPE для нашего игрушечного примера
true = np.array([10, 1009, 0])
pred = np.array([1, 1000, 9])

wmape = weighted_mean_absolute_percentage_error(true, pred)
mape = mean_absolute_percentage_error(true, pred)

print(f'Средняя относительная абсолютная ошибка в % (MAPE): {mape:.2f}%')
print(f'Взвешенная средняя относительная абсолютная ошибка в % (WMAPE): {wmape:.2f}%')

Средняя относительная абсолютная ошибка в % (MAPE): 30000000030.30%
Взвешенная средняя относительная абсолютная ошибка в % (WMAPE): 2.65%


Пример выше наглядно демонстрирует устойчивость метрики к ошибкам на малых значениях, также метрика полезна в бизнес-задачах, где важен оценивать ошибку на общем объёме - прогноза продаж, спроса или логистики.

Но метрика же "взвешенная", где веса в формуле? Их нет, точнее они есть, но они неявные <смайлик> - этот факт зачастую вводит в заблуждения. Объясняю где же спрятались веса:

Если вынести $\tfrac{1}{\sum_{j=1}^n |y_j|}$, то $WMAPE$ можно переписать в виде:


$$WMAPE = \frac{\sum_{i=1}^n |y_i - \hat{y}_i|}{\sum_{j=1}^n |y_j|} = \sum_{i=1}^n \frac{|y_i|}{\sum_{j=1}^n |y_j|} \;\cdot\; \frac{|y_i - \hat{y}_i|}{|y_i|}$$

Тогда получается, что:

$$w_i = \frac{|y_i|}{\sum_{j=1}^n |y_j|} \quad APE_i = \frac{|y_i - \hat{y}_i|}{|y_i|}$$

Следовательно:

$$WMAPE = \sum_{i=1}^n w_i \cdot APE_i \quad \sum_{i=1}^n w_i = 1$$

Получается, что на какжом объекте обучающей выборки мы все также вычисляем уже знакомую нам относительную ошибку величина которой просто взвешивается. 

А можно ли задать свои веса? Да, no problem! Например, когда мы хотим давайть свою важность клиентам, марже, ...

$$
WMAPE_{custom} = \sum_{i=1}^n v_i \cdot APE_i, \quad \sum_{i=1}^n v_i = 1
$$


## Huber Loss

## Квантильная ошибка | Quantile Error/Loss